In [46]:
import PySimpleGUI as sg

import clases.Mapa
import clases.Alineador
import clases.Arbol

arbol = Arbol()
Alineacion = Alineador()
mapa = Mapa()
newick = ''
fasta = ''

layoutSubirArchivo = [  [sg.Text('Filename')],
            [sg.Input(), sg.FileBrowse()], 
            [sg.OK(), sg.Cancel()]]

sg.theme('DarkAmber')   # Add a touch of color
# All the stuff inside your window.
layout = [  [sg.Text('MENU Pongamosle un lindo mensaje')],
            [sg.Button('Subir Fasta')],
            [sg.Text('O ingrese manualmente la secuencia.')],
            [sg.InputText()],
            [sg.Button('Cargar')],
            [sg.Button('Alinear Secuencias')],
            [sg.Button('Armar Arbol')],
            [sg.Button('Mostrar Mapa')],
         ]

# Create the Window
window = sg.Window('Trabajo Final Bioinformatica', layout)
# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED: # if user closes window or clicks cancel
        break
    elif event == 'Subir Fasta':
        event, values = sg.Window('Seleccione un archivo tipo Fasta', layoutSubirArchivo).read(close=True)
    elif event == 'Alinear Secuencias':
        self.alineacion.alinear()
    elif event == 'Armar Arbol':
        self.arbol.armarArbol()
    elif event == 'Mostrar Mapa':
        self.mapa.mostrarMapa()
    print('You entered ', values[0])

window.close()

